In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/Shared\ drives/ecl-pedro/video-coding-utils/notebooks/

AR-CNN-test.ipynb  __init__.py       utils.py          YUV_H264_YUV.sh
checkpoints/       requirements.yml  utils_test.ipynb  yuv_test.ipynb


---

In [1]:
from utils.utils import *
from utils import models

In [3]:
import tensorflow as tf

In [21]:
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.image import psnr

In [5]:
from sklearn.model_selection import train_test_split

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [22]:
def PSNR(x,y):
    return psnr(x,y,max_val=1)

In [37]:
# x: compressed, y: ground truth
x_path = '../../data/H264_raw/BasketballPass_416x240_50_H264_42_Ionly.yuv'
y_path = '../../data/YUV/BasketballPass_416x240_50.yuv'

In [38]:
height = int(y_path.split('_')[1].split('x')[1])
width = int(y_path.split('_')[1].split('x')[0])
size = (height, width)

In [39]:
# select frames
frames = [0]
X = read_YUV420_multiframes(x_path, size, frames, channel='Y')
Y = read_YUV420_multiframes(y_path, size, frames, channel='Y')

In [40]:
X.shape, Y.shape

((1, 240, 416), (1, 240, 416))

In [41]:
# select random patches in those frames
N_patches_per_frame = 5
size_of_patches = (64,64)
idxs = n_random_slices(N_patches_per_frame, range(0,len(frames)), size, size_of_patches)

In [42]:
X = patches_from_idxs(X, idxs)
Y = patches_from_idxs(Y, idxs)

In [43]:
X = X.reshape(*X.shape, 1)
Y = Y.reshape(*Y.shape, 1)

In [44]:
# shapes: (examples, height, width, channels)
X.shape, Y.shape

((5, 64, 64, 1), (5, 64, 64, 1))

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

In [17]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((576, 64, 64, 1), (64, 64, 64, 1), (576, 64, 64, 1), (64, 64, 64, 1))

In [15]:
model = models.QECNN_I()

In [16]:
model.build((None, 64, 64, 1))

In [23]:
model.compile(
    optimizer='SGD',
    loss=MSE,
    metrics=[PSNR]
)

In [24]:
ckpt_callback = ModelCheckpoint(
    filepath='checkpoints/QECNN_I',
    save_weights_only=True,
    verbose=0
)

In [25]:
history = model.fit(
    X_train, Y_train,
    batch_size=128, # (paper=128)
    epochs=100, # guess
    callbacks=[ckpt_callback],
    verbose=2
)

NameError: name 'X_train' is not defined